In [2]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from spotipy import oauth2
import csv
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors
import pandas as pd

In [2]:
cid = '8c4cb3ef1c0c412181ea69d07b3df0ca'
secret = 'a07e05c762a24e12933c4c118c6c18ca'
redirect_uri = 'http://localhost:8910/callback'
username = 'ardabyk07'

In [3]:
#username = input('Enter username: ')
scope = 'user-top-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [3]:
dataset_path = "./data/data.csv"
with open(dataset_path, 'r') as token_file:
    lines = csv.reader(token_file)
    dataset = list(lines)
    data = np.array(dataset)

In [4]:
indices = []
id_index = -1
info = []
choose = -1
name_index = -1
artist_index = -1
for i in range(data.shape[1]):
    if data[0][i] in ['key', 'artists', 'release_date', 'name']:
        indices.append(i)
        if data[0][i] == 'name':
            name_index = i
        if data[0][i] == 'artists':
            artist_index = i
    elif data[0][i] == 'id':
        id_index = i
        
    if data[0][i] in ['artists', 'name']:
        info.append(i)

In [6]:
sourcePlaylistID = '5hkbAFGC5IoFanCLjNVIg3'
sourcePlaylist = sp.user_playlist(username, sourcePlaylistID);
tracks = sourcePlaylist["tracks"];
songs = tracks["items"];

track_id = songs[0]['track']['id']
track_name = songs[0]['track']['name']

audio_features = sp.audio_features(track_id)
selected_song_id = audio_features[0]['id']
selected_song_id

'2b062AChmcZzkeF2kvBMsn'

In [5]:
data = data[1:,:]
np.random.shuffle(data)
ids = np.array(data[:,id_index:id_index+1])

for j in range(data.shape[0]):
    if data[j][id_index] == selected_song_id:
        choose = j
    
ids = np.ndarray.flatten(ids)

song_info = np.array(data[:,info])
indices.append(id_index)
data = np.delete(data, indices, axis=1)

In [6]:
data.shape

(169909, 14)

In [8]:
choose

152989

In [9]:
song_info[choose]

array(["['Johann Sebastian Bach', 'Claudio Arrau']",
       'Goldberg Variations, BWV 988: Aria'], dtype='<U661')

In [10]:
data = data.astype('float64')

In [11]:
#first obtain 5000 songs with NN
model = NearestNeighbors(n_neighbors=5001, algorithm='ball_tree')
scalar = StandardScaler()
scalar.fit(data)
data_nn = scalar.transform(data)
model.fit(data_nn)
distances, indices = model.kneighbors([data_nn[choose]])

In [12]:
selected = song_info[choose]
playlist = song_info[indices]
playlist = playlist[:,1:,:]
indices = np.ndarray.flatten(indices[:,1:])

In [13]:
playlist

array([[["['Johann Sebastian Bach', 'Claudio Arrau']",
         'Goldberg Variations, BWV 988: Aria'],
        ["['Manuel Ponce', 'Andrés Segovia']",
         'Sonatina Meridional p.3'],
        ["['Robert Schumann', 'Vladimir Horowitz']",
         'Träumerei from Kinderszenen, Op. 15'],
        ...,
        ["['Hank Jones']", 'Things Are So Pretty In The Spring'],
        ["['Pat Metheny Group']", 'If I Could'],
        ["['Wolfgang Amadeus Mozart', 'George Szell']",
         'Violin Sonata in C Major, K. 296: II. Andante sostenuto']]],
      dtype='<U661')

In [14]:
data = scalar.transform(data)

In [15]:
tracks = ids[indices]
tracks = np.ndarray.flatten(tracks)
#tracks = list(tracks)

In [16]:
#song_train = []
#found_id = []

tracks_numpy = np.array(tracks)

#found_indices = np.where(np.in1d(ids, tracks_numpy))[0]
#found_id = ids[found_indices]
song_train = data[indices]
#for track_id in tracks:
    #for i, dataset_id in enumerate(ids):
        #if track_id == dataset_id:
            #song_train.append(data[i])
            #found_id.append(track_id)

In [17]:
list(song_info[indices])

[array(["['Johann Sebastian Bach', 'Claudio Arrau']",
        'Goldberg Variations, BWV 988: Aria'], dtype='<U661'),
 array(["['Manuel Ponce', 'Andrés Segovia']", 'Sonatina Meridional p.3'],
       dtype='<U661'),
 array(["['Robert Schumann', 'Vladimir Horowitz']",
        'Träumerei from Kinderszenen, Op. 15'], dtype='<U661'),
 array(["['Robert Schumann', 'Oscar Levant']",
        'Kinderszenen, Op. 15: No. 7, Träumerei (Remastered)'],
       dtype='<U661'),
 array(["['Robert Schumann', 'Vladimir Horowitz']",
        'Träumerei from Kinderszenen, Op. 15'], dtype='<U661'),
 array(["['Robert Schumann', 'Vladimir Horowitz']", 'VII. Träumerei'],
       dtype='<U661'),
 array(["['Ludwig van Beethoven', 'Paul Baumgartner']",
        '33 Piano Variations In C, Op.120 On A Waltz By Anton Diabelli: Variation 30 (Andante, sempre cantabile)'],
       dtype='<U661'),
 array(["['Robert Schumann', 'Vladimir Horowitz']",
        'Von fremden Ländern und Menschen (From "Kinderszenen", Op. 15)'],
    

In [18]:
relevances = []
threshold = 100
relevance = threshold
for i, _ in enumerate(tracks):
    relevances.append(relevance)
    relevance -= 0.0006

In [19]:
relevances

[100,
 99.9994,
 99.99879999999999,
 99.99819999999998,
 99.99759999999998,
 99.99699999999997,
 99.99639999999997,
 99.99579999999996,
 99.99519999999995,
 99.99459999999995,
 99.99399999999994,
 99.99339999999994,
 99.99279999999993,
 99.99219999999993,
 99.99159999999992,
 99.99099999999991,
 99.99039999999991,
 99.9897999999999,
 99.9891999999999,
 99.98859999999989,
 99.98799999999989,
 99.98739999999988,
 99.98679999999987,
 99.98619999999987,
 99.98559999999986,
 99.98499999999986,
 99.98439999999985,
 99.98379999999985,
 99.98319999999984,
 99.98259999999983,
 99.98199999999983,
 99.98139999999982,
 99.98079999999982,
 99.98019999999981,
 99.9795999999998,
 99.9789999999998,
 99.9783999999998,
 99.97779999999979,
 99.97719999999978,
 99.97659999999978,
 99.97599999999977,
 99.97539999999977,
 99.97479999999976,
 99.97419999999975,
 99.97359999999975,
 99.97299999999974,
 99.97239999999974,
 99.97179999999973,
 99.97119999999973,
 99.97059999999972,
 99.96999999999971,
 99.96939

In [20]:
reg = LinearRegression().fit(song_train, relevances)
#for i, track in enumerate(data):
    #if not ids[i] in found_id:
        #predictions.append((reg.predict([track]), ids[i], song_info[i]))
predictions = reg.predict(data)
predicted = np.column_stack((predictions, song_info[:,0], song_info[:,1]))

In [21]:
sort_indices = np.argsort(predictions)[::-1]

In [22]:
predictions = np.array(predictions)
predictions = predictions.astype('float64')
predictions = predictions[sort_indices]

In [23]:
predictions.shape[0]

169909

In [24]:
predicted = predicted[sort_indices]

In [25]:
list(predicted)

[array(['101.55274465676678',
        "['Igor Stravinsky', 'Columbia Symphony Orchestra']",
        'Le sacre du printemps: Part 1 "Adoration of the Earth", The Wise Elder'],
       dtype='<U661'),
 array(['101.55274465676678',
        "['Igor Stravinsky', 'Columbia Symphony Orchestra']",
        'Le sacre du printemps: Part 1 "Adoration of the Earth", The Wise Elder'],
       dtype='<U661'),
 array(['101.37170613636334',
        "['Igor Stravinsky', 'Columbia Symphony Orchestra']",
        'Le sacre du printemps: Part 1 "Adoration of the Earth", The Wise Elder'],
       dtype='<U661'),
 array(['101.10751467518209',
        "['Ludwig van Beethoven', 'Paul Baumgartner']",
        '33 Piano Variations In C, Op.120 On A Waltz By Anton Diabelli: Variation 2 (Poco allegro)'],
       dtype='<U661'),
 array(['101.08366225566523',
        "['Francesco Cilea', 'Orchestra di Torino Della RAI', 'Arturo Basile', 'Coro di Torino della RAI']",
        'L\'Arlesiana, Act II: "Quando la luce muor" (Vo

In [26]:
first_threshold_index = 0
for i in range(predictions.shape[0]):
    if predictions[i] <= threshold + 1:
        first_threshold_index = i
        break
print(first_threshold_index)
playlist_added = ids[sort_indices][first_threshold_index:first_threshold_index+100]

6


In [27]:
playlist_name = "CS464 Term Project (Linear Regression) - Playlist based on " + str(track_name)
scope = "playlist-modify-public"
#username = input('Enter username:')
token = util.prompt_for_user_token(username,scope,client_id=cid,client_secret=secret,redirect_uri=redirect_uri) 
sp = spotipy.Spotify(auth=token)
sp.user_playlist_create(username, name=playlist_name)
playlists = sp.user_playlists(username)
sp.user_playlist_add_tracks(username, playlists['items'][0]['id'], list(playlist_added))

{'snapshot_id': 'MixmN2MyM2I1YjBkZjlhMGQxZjkxNzZiMGMxN2ZkYmVhMWIyZTMxNzZi'}